In [11]:
import pandas as pd
import numpy as np 

### Data Collection

In [12]:
import requests

# URLs of the files
train_data_url = 'https://www.raphaelcousin.com/modules/data-science-practice/module3/exercise/module3_exercise_train.csv'
test_data_url = 'https://www.raphaelcousin.com/modules/data-science-practice/module3/exercise/module3_exercise_test.csv'

# Function to download a file
def download_file(url, file_name):
    response = requests.get(url)
    response.raise_for_status()  # Ensure we notice bad responses
    with open(file_name, 'wb') as file:
        file.write(response.content)
    print(f'Downloaded {file_name} from {url}')

# Downloading the files
download_file(train_data_url, 'module3_exercise_train.csv')
download_file(test_data_url, 'module3_exercise_test.csv')

Downloaded module3_exercise_train.csv from https://www.raphaelcousin.com/modules/data-science-practice/module3/exercise/module3_exercise_train.csv
Downloaded module3_exercise_test.csv from https://www.raphaelcousin.com/modules/data-science-practice/module3/exercise/module3_exercise_test.csv


In [13]:
df_train =  pd.read_csv("module3_exercise_train.csv", sep=",", index_col='id')
df_train.head()

,BedroomAbvGr,KitchenAbvGr,TotRmsAbvGrd,Fireplaces,GarageYrBlt,GarageCars,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SalePrice
id,,,,,,,,,,,,,,,,,
254,2,1,4.0,1,1977.0,two,440,0,55,0,0,165,0,0,7,2010.0,127500
1066,1,1,5.0,1,1983.0,two,612,349,40,0,0,0,0,0,9,2009.0,316600
638,4,1,10.0,1,1998.0,two,420,144,123,0,0,0,0,0,7,2006.0,258000
799,3,1,8.0,0,1916.0,one,180,0,0,0,140,0,0,0,8,2009.0,135000
380,2,1,5.0,0,2005.0,two,438,108,0,0,0,0,0,0,3,2006.0,167240


### Data Preprocessing

In [14]:

print(df_train.columns)
df_train.drop(columns=[])
df_train.fillna(-1)

df_train.head()

Index(['BedroomAbvGr', 'KitchenAbvGr', 'TotRmsAbvGrd', 'Fireplaces',
       'GarageYrBlt', 'GarageCars', 'GarageArea', 'WoodDeckSF', 'OpenPorchSF',
       'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea', 'MiscVal',
       'MoSold', 'YrSold', 'SalePrice'],
      dtype='object')


,BedroomAbvGr,KitchenAbvGr,TotRmsAbvGrd,Fireplaces,GarageYrBlt,GarageCars,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SalePrice
id,,,,,,,,,,,,,,,,,
254,2,1,4.0,1,1977.0,two,440,0,55,0,0,165,0,0,7,2010.0,127500
1066,1,1,5.0,1,1983.0,two,612,349,40,0,0,0,0,0,9,2009.0,316600
638,4,1,10.0,1,1998.0,two,420,144,123,0,0,0,0,0,7,2006.0,258000
799,3,1,8.0,0,1916.0,one,180,0,0,0,140,0,0,0,8,2009.0,135000
380,2,1,5.0,0,2005.0,two,438,108,0,0,0,0,0,0,3,2006.0,167240


### Model Building and Evaluation

In [15]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline


TARGET = "SalePrice"

# Charger le test d'abord
X_test = pd.read_csv("module3_exercise_test.csv", sep=",", index_col="id")

# Features = intersection train/test (hors cible)
feat = [c for c in df_train.columns if c != TARGET and c in X_test.columns]

# Données numériques
X_tr = df_train[feat].apply(pd.to_numeric, errors="coerce")
y_tr = df_train[TARGET].astype(float)
X_te = X_test[feat].apply(pd.to_numeric, errors="coerce")

# Pipeline et fit
pipe = Pipeline([("imputer", SimpleImputer(strategy="median")),
                 ("model", LinearRegression())])
pipe.fit(X_tr, y_tr)


/home/antoine/Documents/sorbonne/td_ds/data_science_practice_2025/.venv/lib/python3.10/site-packages/sklearn/impute/_base.py:653: UserWarning: Skipping features without any observed values: ['GarageCars']. At least one non-missing value is needed for imputation with strategy='median'.
  warnings.warn(


,steps,"[('imputer', ...), ('model', ...)]"
,transform_input,None
,memory,None
,verbose,False
,missing_values,nan
,strategy,'median'
,fill_value,None
,copy,True
,add_indicator,False
,keep_empty_features,False
,fit_intercept,True


### Generating Submission File

In [16]:
y_pred = pipe.predict(X_te)
assert len(y_pred) == len(X_test)


/home/antoine/Documents/sorbonne/td_ds/data_science_practice_2025/.venv/lib/python3.10/site-packages/sklearn/impute/_base.py:653: UserWarning: Skipping features without any observed values: ['GarageCars']. At least one non-missing value is needed for imputation with strategy='median'.
  warnings.warn(


In [17]:
submission = pd.DataFrame({
    'id': X_test.index,
    'SalePrice': y_pred,
})

submission.to_csv('submission.csv', index=False, sep=',')
submission.head()

,id,SalePrice
0,892,269677.361330
1,1105,234540.704948
2,413,261983.158492
3,522,224159.318654
4,1036,150640.671186
